In [1]:
import pandas as pd

In [4]:
df = pd.read_csv('/home/aaron/Downloads/Download(2).csv' , encoding = 'big5' , header = 1 , on_bad_lines='skip')

In [5]:
df

,帳單結帳日：110/09/15 繳款截止日（遇假日順延）：110/10/05/請儘速繳款
0,帳務資訊：
1,帳單資訊：
2,紅利點數：
3,您本月各幣別消費明細如下：
4,新臺幣：
5,美元：


In [3]:
import requests
from io import StringIO
import pandas as pd



In [4]:
from datetime import date
today = date.today()
d1 = today.strftime("%Y%m%d")
d1

'20220912'

In [5]:
date = '20220824'
r = requests.get('http://www.tse.com.tw/fund/T86?response=csv&date='+date+'&selectType=ALLBUT0999')

df = pd.read_csv(StringIO(r.text) , header = 1).dropna(how='all' ,axis =1 ).dropna(how='any')

In [6]:
df

,證券代號,證券名稱,外陸資買進股數(不含外資自營商),外陸資賣出股數(不含外資自營商),外陸資買賣超股數(不含外資自營商),外資自營商買進股數,外資自營商賣出股數,外資自營商買賣超股數,投信買進股數,投信賣出股數,投信買賣超股數,自營商買賣超股數,自營商買進股數(自行買賣),自營商賣出股數(自行買賣),自營商買賣超股數(自行買賣),自營商買進股數(避險),自營商賣出股數(避險),自營商買賣超股數(避險),三大法人買賣超股數
0,"=""00715L""",期街口布蘭特正2,"1,075,000","920,000","155,000",0.0,0.0,0.0,0,0,0,"23,974,000",0,0,0,"24,872,000","898,000","23,974,000","24,129,000"
1,5258,虹堡,"12,927,000","4,777,370","8,149,630",0.0,0.0,0.0,0,0,0,"-8,000",0,"8,000","-8,000",0,0,0,"8,141,630"
2,"=""00642U""",期元大S&P石油,"37,000","37,000",0,0.0,0.0,0.0,0,0,0,"5,417,099",0,0,0,"5,753,099","336,000","5,417,099","5,417,099"
3,2867,三商壽,"5,220,000","360,000","4,860,000",0.0,0.0,0.0,0,0,0,"14,000","1,000",0,"1,000","13,000",0,"13,000","4,874,000"
4,2838,聯邦銀,"5,757,000","1,374,000","4,383,000",0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,"4,383,000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1101,"=""00882""",中信中國高股息,"520,000","665,000","-145,000",0.0,0.0,0.0,0,0,0,"-17,764,362",0,0,0,"561,000","18,325,362","-17,764,362","-17,909,362"
1102,"=""0056""",元大高股息,"916,102","896,000","20,102",0.0,0.0,0.0,0,0,0,"-19,900,704","50,000",0,"50,000","733,999","20,684,703","-19,950,704","-19,880,602"
1103,2887,台新金,"23,843,501","46,082,037","-22,238,536",0.0,0.0,0.0,"18,000",0,"18,000","-1,351,827","281,000","189,586","91,414","676,000","2,119,241","-1,443,241","-23,572,363"
1104,"=""00673R""",期元大S&P原油反1,"433,000","254,000","179,000",0.0,0.0,0.0,0,0,0,"-28,567,771",0,0,0,"2,087,000","30,654,771","-28,567,771","-28,388,771"


In [4]:
pd.read_csv(StringIO(r.text) , header = 1).dropna(how='all' , axis =1 ).dropna(how='any')

,證券代號,證券名稱,外陸資買進股數(不含外資自營商),外陸資賣出股數(不含外資自營商),外陸資買賣超股數(不含外資自營商),外資自營商買進股數,外資自營商賣出股數,外資自營商買賣超股數,投信買進股數,投信賣出股數,投信買賣超股數,自營商買賣超股數,自營商買進股數(自行買賣),自營商賣出股數(自行買賣),自營商買賣超股數(自行買賣),自營商買進股數(避險),自營商賣出股數(避險),自營商買賣超股數(避險),三大法人買賣超股數
0,"=""00715L""",期街口布蘭特正2,"1,075,000","920,000","155,000",0.0,0.0,0.0,0,0,0,"23,974,000",0,0,0,"24,872,000","898,000","23,974,000","24,129,000"
1,5258,虹堡,"12,927,000","4,777,370","8,149,630",0.0,0.0,0.0,0,0,0,"-8,000",0,"8,000","-8,000",0,0,0,"8,141,630"
2,"=""00642U""",期元大S&P石油,"37,000","37,000",0,0.0,0.0,0.0,0,0,0,"5,417,099",0,0,0,"5,753,099","336,000","5,417,099","5,417,099"
3,2867,三商壽,"5,220,000","360,000","4,860,000",0.0,0.0,0.0,0,0,0,"14,000","1,000",0,"1,000","13,000",0,"13,000","4,874,000"
4,2838,聯邦銀,"5,757,000","1,374,000","4,383,000",0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,"4,383,000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1101,"=""00882""",中信中國高股息,"520,000","665,000","-145,000",0.0,0.0,0.0,0,0,0,"-17,764,362",0,0,0,"561,000","18,325,362","-17,764,362","-17,909,362"
1102,"=""0056""",元大高股息,"916,102","896,000","20,102",0.0,0.0,0.0,0,0,0,"-19,900,704","50,000",0,"50,000","733,999","20,684,703","-19,950,704","-19,880,602"
1103,2887,台新金,"23,843,501","46,082,037","-22,238,536",0.0,0.0,0.0,"18,000",0,"18,000","-1,351,827","281,000","189,586","91,414","676,000","2,119,241","-1,443,241","-23,572,363"
1104,"=""00673R""",期元大S&P原油反1,"433,000","254,000","179,000",0.0,0.0,0.0,0,0,0,"-28,567,771",0,0,0,"2,087,000","30,654,771","-28,567,771","-28,388,771"


In [5]:
df = df.astype(str).apply(lambda s: s.str.replace('"' , ''))

In [6]:
df['stock_id'] = df['證券代號'].str.replace('=' , '').str.replace('"' , '')

In [7]:
df

,證券代號,證券名稱,外陸資買進股數(不含外資自營商),外陸資賣出股數(不含外資自營商),外陸資買賣超股數(不含外資自營商),外資自營商買進股數,外資自營商賣出股數,外資自營商買賣超股數,投信買進股數,投信賣出股數,投信買賣超股數,自營商買賣超股數,自營商買進股數(自行買賣),自營商賣出股數(自行買賣),自營商買賣超股數(自行買賣),自營商買進股數(避險),自營商賣出股數(避險),自營商買賣超股數(避險),三大法人買賣超股數,stock_id
0,=00715L,期街口布蘭特正2,"1,075,000","920,000","155,000",0.0,0.0,0.0,0,0,0,"23,974,000",0,0,0,"24,872,000","898,000","23,974,000","24,129,000",00715L
1,5258,虹堡,"12,927,000","4,777,370","8,149,630",0.0,0.0,0.0,0,0,0,"-8,000",0,"8,000","-8,000",0,0,0,"8,141,630",5258
2,=00642U,期元大S&P石油,"37,000","37,000",0,0.0,0.0,0.0,0,0,0,"5,417,099",0,0,0,"5,753,099","336,000","5,417,099","5,417,099",00642U
3,2867,三商壽,"5,220,000","360,000","4,860,000",0.0,0.0,0.0,0,0,0,"14,000","1,000",0,"1,000","13,000",0,"13,000","4,874,000",2867
4,2838,聯邦銀,"5,757,000","1,374,000","4,383,000",0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,"4,383,000",2838
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1101,=00882,中信中國高股息,"520,000","665,000","-145,000",0.0,0.0,0.0,0,0,0,"-17,764,362",0,0,0,"561,000","18,325,362","-17,764,362","-17,909,362",00882
1102,=0056,元大高股息,"916,102","896,000","20,102",0.0,0.0,0.0,0,0,0,"-19,900,704","50,000",0,"50,000","733,999","20,684,703","-19,950,704","-19,880,602",0056
1103,2887,台新金,"23,843,501","46,082,037","-22,238,536",0.0,0.0,0.0,"18,000",0,"18,000","-1,351,827","281,000","189,586","91,414","676,000","2,119,241","-1,443,241","-23,572,363",2887
1104,=00673R,期元大S&P原油反1,"433,000","254,000","179,000",0.0,0.0,0.0,0,0,0,"-28,567,771",0,0,0,"2,087,000","30,654,771","-28,567,771","-28,388,771",00673R


In [2]:
import pandas as pd
import numpy as np
from datetime import date
import requests
from io import StringIO

today = date.today()

In [23]:
date = today.strftime("%Y%m%d")
date

'20220907'

In [40]:
r = requests.get('http://www.tse.com.tw/fund/T86?response=csv&date='+date+'&selectType=ALLBUT0999')
df = pd.read_csv(StringIO(r.text) , header = 1).dropna(how='all' ,axis =1 ).dropna(how='any')
            #移除原先表格中會有的多餘符號
df = df.astype(str).apply(lambda s: s.str.replace('"' , ''))
# df['證券代號'].str.replace('=' , '').str.replace('"' , '')
df['股票名稱'] = df['證券名稱'].str.replace('=' , '').str.replace('"' , '')

In [41]:
df = df.drop(['證券名稱'] , axis=1)

In [42]:
df = df.set_index(['股票名稱'])

In [44]:
print(df)

                   證券代號 外陸資買進股數(不含外資自營商) 外陸資賣出股數(不含外資自營商) 外陸資買賣超股數(不含外資自營商)  \
股票名稱                                                                          
元大台灣50反1        =00632R        8,516,000        3,631,000         4,885,000   
期元大S&P原油反1      =00673R        1,027,000        1,707,000          -680,000   
國泰臺灣加權反1        =00664R                0           47,000           -47,000   
元大滬深300正2       =00637L        3,278,000        1,090,000         2,188,000   
群創                 3481       23,101,010       16,387,000         6,714,010   
...                 ...              ...              ...               ...   
新光金                2888        9,410,576       26,670,067       -17,259,491   
台新金                2887        7,851,083       29,901,940       -22,050,857   
元大高股息             =0056          520,913        2,768,000        -2,247,087   
期街口布蘭特正2        =00715L        1,799,000        7,050,000        -5,251,000   
國泰永續高股息          =00878        2,051,692       22,35

In [22]:
df.apply(lambda s: pd.to_numeric(s , errors = 'coerce'))

,證券名稱,外陸資買進股數(不含外資自營商),外陸資賣出股數(不含外資自營商),外陸資買賣超股數(不含外資自營商),外資自營商買進股數,外資自營商賣出股數,外資自營商買賣超股數,投信買進股數,投信賣出股數,投信買賣超股數,自營商買賣超股數,自營商買進股數(自行買賣),自營商賣出股數(自行買賣),自營商買賣超股數(自行買賣),自營商買進股數(避險),自營商賣出股數(避險),自營商買賣超股數(避險),三大法人買賣超股數,股票名稱,date
0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220907
1,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,20220907
2,NaN,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,20220907
3,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,20220907
4,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3481.0,20220907
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1121,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2888.0,20220907
1122,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2887.0,20220907
1123,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56.0,20220907
1124,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,20220907


In [7]:
from datetime import datetime
from threading import Timer

In [9]:
x = datetime.today()
x

datetime.datetime(2022, 9, 13, 1, 0, 20, 597071)

In [10]:
y = x.replace(day = x.day+1 , hour = 1 , minute = 0 , second = 0 , microsecond= 0)


In [22]:

import schedule
def job():
    print("I'm working...")
# schedule.every(10).minutes.do(job)
schedule.every().day.at("01:08").do(job)

Every 1 day at 01:08:00 do job() (last run: [never], next run: 2022-09-13 01:08:00)